### Intervals analysis with fixed on-time

In [1]:
using Stripeline
using Plots
using Healpix
using Statistics
using LaTeXStrings

In [2]:
function strip_map(time_range, map)
    dirs, _=genpointings(telescope_motors, Float64[0, 0, 1], time_samples_1)   #We only need the direction so we omit the second return value
    sp=0
    for k in 1:length(time_samples_1)
        colat, long=dirs[k, :]    #Extract the colatitude and the longitude from "dirs"
        pixel_index=ang2pix(map, colat, long)
        map[pixel_index]=1    #we are not interested in understanding the pixels covered at least once, we don't need to know how many times the telescope covers every pixel
    end
    for i in 1:length(mapp.pixels)
        if mapp[i]>=1
            sp+=1
        end
        i+=1
    end
    return sp
end

function period_tod(zeros, counts, durations, n, sampling)    #This fill a single period of the tod
    m=0
    while m<durations[2]    #m is a minute of the period, i is the index governing the tod
        if m<durations[1]    #It means we are in the on part of the period
            v=1
            c=1
        end
        if m>=durations[1] && m<durations[2]    #It means we are in the off part of the period
            v=0
            c=2
        end
        b=((n*durations[2])+m)*sampling
        for a=1:sampling    #A complete cicle corresponds to an entire minute of the period which corresponds to sampling components of the tod
            if (b+a)<=length(zeros)
                zeros[b+a]=v
                counts[c]+=1
                a+=1
            end
            if (b+a)>length(zeros)
                a=sampling+10
                m=durations[2]+10
            end
        end
        m+=1
    end
end

function fill_tod(zeros, counts, durations, periods, sampling, y)    #Through period_tod this fill the entire tod
    #For each moment in wich the calibrator is on the tod's value should be 1 instead of 0, at the beginning of the experiment the calibrator is on. During the first minute of each ignition the tod's value will be 0.5 so that we can distinguish it from the following nine
    for n=0:periods    #We fill each period separately using another function
        period_tod(zeros, counts, durations, n, sampling)
        n+=1
    end
end

function project_to_map(time_samples, map, tod)
    telescope_motors(time_s)=(0.0, deg2rad(20.0), timetorotang(time_s, 1))
    dirs, _=genpointings(telescope_motors, Float64[0, 0, 1], time_samples)   #We only need the direction so we omit the second return value
    for k in 1:length(time_samples)
        colat, long=dirs[k, :]    #Extract the colatitude and the longitude from "dirs"
        pixel_index=ang2pix(map, colat, long)
        if tod[k]>0    #This analisys aim to study the calibration process so we only use the stable calibrator's ignitions
            map[pixel_index]+=1    #Every time a pixel is covered by the calibrator's signal, it's colour value is increased by a constant value This way we can both see wich parts of the sky are covered and how many times the calibrator obscures them.  
        end
        if tod[k]==0 && map[pixel_index]==0
            map[pixel_index]=0.5   #This way we can distinguish the pixels in the strip visible to the telescope from the ones out of it
        end
        if map[pixel_index]>0.5 && (map[pixel_index]%1)!=0
            map[pixel_index]-=0.5   #We want the pixel_index to represents the number of calibrator's passages over a pixel, so we don't want our method to identify the strip to ruin this physical meaning
        end
    end
end

function coveradge_counter(map, pixels, hit_count, tot_count_diff, ideals)    #This fills the arrays keeping track of the hit count of the considered pixels
    j=1    #We need an index to controll the filling array
    for r in 1:length(pixels)
        if map[r]==0.5    #This pixel has not been covered by the calibrator's signal so we put it in the strip pixels array but whit its real count null
            hit_count[j]=0
            tot_count_diff[j]=-ideals
            j+=1    #We increment this index only when the hit count is >0
        end
        if map[r]>=1
            hit_count[j]=map[r]
            tot_count_diff[j]=hit_count[j]-ideals
            j+=1
        end
        r+=1
    end
end

coveradge_counter (generic function with 1 method)

In [3]:
ni=24    #Number of ignitions (fixed)
cal_duration=10    #Stable ignition time expressed in minutes
frequency=50    #Sampling frequency in Hz i.e. number of samples per seconds
sampling=frequency*60    #Samples per minute i.e. sampling frequency * 60 seconds
#We run a first simulation without the calibrator to understand the minimum time needed by the telescope to cover completely the sky strip
time_samples_1=0.0:1/frequency:(60*24*60)    #Total time set to 24 hours
telescope_motors(time_s)=(0.0, deg2rad(20.0), timetorotang(time_s, 1))
mapp=Healpix.HealpixMap{Float64, RingOrder}(256)   #We create a map wich represents the whole sky
strip_pixels=strip_map(time_samples_1, mapp)    #Here we run the simulation
on_samples=ni*cal_duration*frequency*60    #Total time, expressed in seconds sampled at 50 Hz, during which the calibrator is on. This is fixed now.
ideal_count=on_samples/strip_pixels    #If all the pixels were seen the same number of times this would be their hit count
#We define the arrays useful later
cal_intervals=50:1:190    #We define an array of values we'll use as time between to ignitions to compare them, here we express them in minutes
dev_sts_tot=zeros(Float64, length(cal_intervals), 1)    #We'll define the associated mean value using the number of pixels in the entire Strip, we only evaluate this standard deviation

141×1 Matrix{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [4]:
for ind in 1:length(cal_intervals)
    #We define the tod wich will be used to simulate the calibrator
    obs_time=ni*(cal_intervals[ind]+cal_duration)    #Total observational time in minutes
    time_samples=0.0:(1/frequency):(obs_time*60)
    #We define the tod wich will be used to simulate the calibrator
    counts=[0, 0]    #Onche filled the array is defined as follows. First component: number of samples during which the calibrator is on, unstable or not. Second component: number of samples during which the calibrator is off.
    durations=[cal_duration, cal_intervals[ind]+cal_duration]    #This array represents the division of a period of the calibrator's action expressed in minutes
    tod=zeros(Float64, length(time_samples), 1)    #The entire tod is initialized to zero
    n_per=cld(obs_time, durations[2])    #Number of periods in which the total time can be divided rounded to the maximum integer possible  
    fill_tod(tod, counts, durations, n_per, sampling, 0)    #We fill the tod. When the value is 0.5 the calibrator is on but unstable, when it equals 1 the calibrator is on and stable, if the calibrator is off the value remains 0.   
    #Now we work on the standard deviation
    #We now build the map (without plotting it)
    map=Healpix.HealpixMap{Float64, RingOrder}(256)   #We create a map wich represents the whole sky
    project_to_map(time_samples, map, tod)    #Here we run the simulation
    #We need to evaluate in a quantitative way the uniformity of the calibrator signal's coverage so we can start by studying map.pixels i.e. the array containing the count of the calibrator's coverages af each pixel
    #We define and fill three arrays storing only the data we need to make the real analysis
    hit_count=zeros(Float64, strip_pixels, 1)    #The hit count of every pixel in the strip
    tot_count_diff=zeros(Float64, strip_pixels, 1)    #The difference between the real count of every pixel in the strip and the ideal value
    coveradge_counter(map, map.pixels, hit_count, tot_count_diff, ideal_count)   #Here we fill the arrays previuosly created
    #We adjourn the final array
    dev_sts_tot[ind]=stdm(tot_count_diff, ideal_count, corrected=true)    #Standard deviation considering the whole strip
    println("Component ", ind, " of ", length(cal_intervals), " has been processed.")
    ind+=1
end

Component 1 of 141 has been processed.
Component 2 of 141 has been processed.
Component 3 of 141 has been processed.
Component 4 of 141 has been processed.
Component 5 of 141 has been processed.
Component 6 of 141 has been processed.
Component 7 of 141 has been processed.
Component 8 of 141 has been processed.
Component 9 of 141 has been processed.
Component 10 of 141 has been processed.
Component 11 of 141 has been processed.
Component 12 of 141 has been processed.
Component 13 of 141 has been processed.
Component 14 of 141 has been processed.
Component 15 of 141 has been processed.
Component 16 of 141 has been processed.
Component 17 of 141 has been processed.
Component 18 of 141 has been processed.
Component 19 of 141 has been processed.
Component 20 of 141 has been processed.
Component 21 of 141 has been processed.
Component 22 of 141 has been processed.
Component 23 of 141 has been processed.
Component 24 of 141 has been processed.
Component 25 of 141 has been processed.
Component

In [6]:
theme(:default)
plot(cal_intervals, dev_sts_tot, xlabel=L"t_\mathrm{off}\;[\mathrm{min}]", ylabel=L"\sigma_\mathrm{tot}", minorgrid=true, markershape=:circle, markersize=2, linewidth=0.8, linealpha=0.6, legend=:false)
savefig("plot_sdt_ver2.pdf")

"C:\\Users\\user\\Tesi triennale\\plot_sdt_ver2.pdf"